<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train8_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:

train8 = pd.read_csv(r'/content/train_data8.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [ ]:

train8['Target'] = train8['Target'].replace(['Others'],'Invalid')


In [ ]:
train8['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [ ]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train8_Xtfidf = tfidf_vect.fit_transform(train8['Sentence'])


In [ ]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [ ]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [ ]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [ ]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [ ]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [ ]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train8_Xtfidf,train8['Target'],test_size=0.26,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.665929203539823
                  precision    recall  f1-score   support

        Analysis       0.48      0.33      0.39       127
      Conclusion       0.85      0.38      0.52        45
           Facts       0.72      0.89      0.80       486
         Invalid       0.52      0.55      0.54       130
           Issue       0.58      0.39      0.46        49
Rule/Law/Holding       0.67      0.30      0.41        67

        accuracy                           0.67       904
       macro avg       0.64      0.47      0.52       904
    weighted avg       0.66      0.67      0.64       904



In [ ]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 3, 3, 2, 2, 1, 3, 0, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 0, 2, 2, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2]
[0.9978721637760471, 0.9844504878752948, 0.9985359585493387, 0.9988017841334668, 0.9890484216130943, 0.9957135428038592, 0.998258254530638, 0.9873797199244184, 0.9981145965853914, 0.9866546649949748, 0.99657623170529, 0.9970908966029358, 0.9951804735533547, 0.9982963603059632, 0.9987205831136814, 0.9984956857658157, 0.9980056646255034, 0.9883351046611586, 0.9987357240802024, 0.99584353072627, 0.9982086666094744, 0.9974456186000736, 0.9979751061900155, 0.9943885465785454, 0.994736862035534, 0.98680744306529, 0.9929228064959641, 0.9929147772017994, 0.9943120238232909, 0.9857769510923199, 0.9927379117534233, 0.9976130337463516, 0.9812811212351003, 0.9958963239160278, 0.9975372951772105, 0.9889756073009548, 0.9978526797632166, 0.9940922988165186, 0.9983273950569215, 0.9946610460211825, 0.9907705486427105, 0.9986479412

In [ ]:
unlabel_1=unlabel_1.loc[pos,:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
2,525,85 22 sw 140 wherein accused acting upon advic...,Rule/Law/Holding
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
6,324,defendant argues impossible reconcile plaintif...,Facts


In [ ]:
frame_1 = [train8,unlabel_1]
train8_1 = pd.concat(frame_1)
len(train8_1)

3534

In [ ]:
x_train_1 = tfidf_vect.transform(train8_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train8_1['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6562942008486563
                  precision    recall  f1-score   support

        Analysis       0.40      0.34      0.37        86
      Conclusion       0.70      0.44      0.54        36
           Facts       0.74      0.85      0.79       380
         Invalid       0.54      0.59      0.57       115
           Issue       0.63      0.41      0.50        46
Rule/Law/Holding       0.47      0.20      0.29        44

        accuracy                           0.66       707
       macro avg       0.58      0.47      0.51       707
    weighted avg       0.64      0.66      0.64       707



In [ ]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 5, 3, 1, 2, 3, 3, 2, 3, 1, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 2, 3, 2, 2, 2, 2, 2, 5, 3, 2]
[0.9986669079509023, 0.9990986070165074, 0.9932541914690428, 0.9980366010512653, 0.9986839929123453, 0.997952968914933, 0.9987641694182711, 0.997527274918181, 0.9845328982682091, 0.9988122469191881, 0.9903071151555053, 0.9954274496696843, 0.9924515451356236, 0.9966802517596891, 0.9975342107645291, 0.9832335270576978, 0.9908571206342992, 0.9817006761456506, 0.9990522817878529, 0.987198113901542, 0.9980651702000958, 0.9961127273566464, 0.9986396371276678, 0.9991605102938372, 0.996691530025195, 0.9972727665536488, 0.9960131002205767, 0.9935014875501055, 0.9867127208818941, 0.9988698009470243, 0.9982023430635588, 0.9873416688562897, 0.9989747940006166, 0.9989455876539552, 0.9988080608203441, 0.9990080184647904, 0.9905730886346313, 0.9971596626147256, 0.9965101216614686, 0.9979817864977025, 0.9924590268056945, 0.9979730495745706, 0.9924

In [ ]:
unlabel_2 = unlabel_2.loc[pos,:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
5,2278,evidence state appellant shows appellant alleg...,Analysis
6,570,appellant performed work asked 7500 money ther...,Facts
7,3150,finding door facing street seen man walking pr...,Facts


In [ ]:
frame_2 = [train8_1,unlabel_2]
train8_2 = pd.concat(frame_2)
len(train8_2)

3586

In [ ]:
x_train_2 = tfidf_vect.transform(train8_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train8_2['Target'],test_size=0.27,random_state=2)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.631578947368421
                  precision    recall  f1-score   support

        Analysis       0.52      0.32      0.40       151
      Conclusion       0.63      0.46      0.54        41
           Facts       0.66      0.90      0.76       474
         Invalid       0.56      0.46      0.51       163
           Issue       0.87      0.38      0.53        68
Rule/Law/Holding       0.53      0.24      0.33        72

        accuracy                           0.63       969
       macro avg       0.63      0.46      0.51       969
    weighted avg       0.62      0.63      0.60       969



In [ ]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 3, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 1, 2, 5, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 0, 2, 2, 4, 2, 2, 2, 2, 2]
[0.995381916134994, 0.9945663632623193, 0.9970938979884796, 0.9974924599718419, 0.9970204802449214, 0.9978363305952515, 0.9980401774549705, 0.9860202066015826, 0.9981751436547086, 0.9984563650943981, 0.9984294445306076, 0.9980483709076197, 0.9934608257311605, 0.998050783162001, 0.9905548635745854, 0.9981157506923004, 0.9977933025607558, 0.9983117601566641, 0.996729480423584, 0.989284663267921, 0.998097887989136, 0.9972113220620452, 0.9980883176742118, 0.9982005702168287, 0.9968476906045447, 0.9950460811039854, 0.9918733437010234, 0.9944970763206563, 0.9967325266567895, 0.9973603148017983, 0.9847138288481475, 0.9984648609268878, 0.9981728413946919, 0.9950992651779936, 0.9981241970013849, 0.9964212668798303, 0.9976775051821352, 0.9928378666260217, 0.9923721499051843, 0.9982380027310126, 0.995737731941

In [ ]:
unlabel_3=unlabel_3.loc[pos,:]
frame_3 = [train8_2,unlabel_3]
train8_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train8_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train8_3['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6438356164383562
                  precision    recall  f1-score   support

        Analysis       0.45      0.30      0.36        98
      Conclusion       0.65      0.41      0.50        37
           Facts       0.68      0.89      0.77       377
         Invalid       0.59      0.50      0.54       117
           Issue       0.62      0.40      0.49        50
Rule/Law/Holding       0.60      0.24      0.34        51

        accuracy                           0.64       730
       macro avg       0.60      0.45      0.50       730
    weighted avg       0.63      0.64      0.62       730



In [ ]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 2, 3, 2, 5, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 0, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 3, 0, 2, 2, 2, 5, 3, 0, 2, 2, 2, 2, 0, 2, 2, 3, 1, 2, 2, 2, 3, 2, 2, 2, 2, 5, 2, 2, 2, 2]
[0.9962311561274758, 0.9831250347119743, 0.9916367483653887, 0.9978453745202178, 0.9903823962783893, 0.9987437906702956, 0.9934080879380179, 0.9912207458218216, 0.9987879303897285, 0.9986486722622084, 0.9925421082415047, 0.9985160629991113, 0.997720426737739, 0.9941257157254922, 0.9987713190916518, 0.9988505461758909, 0.9975881868668698, 0.9984170590301169, 0.9920406253386449, 0.9902702738354036, 0.9911424354515961, 0.9985290062460499, 0.9987540645897147, 0.9987936923694342, 0.9989091185116474, 0.9984505771953626, 0.9983797233151241, 0.998615303953958, 0.9977652115811187, 0.9985325722636521, 0.993667702159738, 0.9844965809465726, 0.9963746160064885, 0.9841425233731196, 0.9907637940424844, 0.9954262475839761, 0.9976615284518402, 0.9930054510995099, 0.9938859407784961, 0.9975163741912038, 0.9

In [ ]:
unlabel_4=unlabel_4.loc[pos,:]

In [ ]:
frame_4 = [train8_3,unlabel_4]
train8_4 = pd.concat(frame_4)
len(train8_4)
x_train_4 = tfidf_vect.transform(train8_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train8_4['Target'],test_size=0.27,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6723107569721115
                  precision    recall  f1-score   support

        Analysis       0.54      0.33      0.41       138
      Conclusion       0.75      0.54      0.63        56
           Facts       0.72      0.88      0.79       536
         Invalid       0.57      0.58      0.57       151
           Issue       0.67      0.40      0.50        60
Rule/Law/Holding       0.48      0.25      0.33        63

        accuracy                           0.67      1004
       macro avg       0.62      0.50      0.54      1004
    weighted avg       0.66      0.67      0.65      1004



In [ ]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 2, 3, 3, 2, 3, 1, 2, 3, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 0, 0, 2, 2, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 2, 1, 2, 2, 3, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 0, 3, 3, 2]
[0.9943257642690735, 0.9988928435062013, 0.9971806622616386, 0.9990243514089013, 0.99357339946349, 0.9990243514089013, 0.9974979163325224, 0.9988689088325647, 0.9990544757940791, 0.9989972031327038, 0.9986887684109587, 0.9985631125801085, 0.9923082870978852, 0.9923996549671202, 0.9970059765481344, 0.9954791840597448, 0.9941602090931629, 0.994938274481817, 0.9959771692468778, 0.9983748445538254, 0.9990256827814763, 0.9968323609902715, 0.9966260593095194, 0.9917258443627748, 0.9979945678886033, 0.9939995854379975, 0.9939735686642982, 0.9934616900406847, 0.9812717400369181, 0.9954438967710836, 0.9973234209204588, 0.9988170344460618, 0.9985608891303195, 0.9989431527218727, 0.9990243514089013, 0.9974200698841128, 0.9987027424399626, 0.9894930506734969, 0.997550031224135, 0.9986732450186

In [ ]:
unlabel_5 = unlabel_5.loc[pos,:]

In [ ]:
frame_5 = [train8_4,unlabel_5]
train8_5 = pd.concat(frame_5)
len(train8_5)
x_train_5 = tfidf_vect.transform(train8_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train8_5['Target'],test_size=0.27,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6741682974559687
                  precision    recall  f1-score   support

        Analysis       0.50      0.31      0.39       150
      Conclusion       0.85      0.50      0.63        58
           Facts       0.72      0.90      0.80       543
         Invalid       0.56      0.58      0.57       145
           Issue       0.61      0.42      0.49        55
Rule/Law/Holding       0.67      0.25      0.37        71

        accuracy                           0.67      1022
       macro avg       0.65      0.49      0.54      1022
    weighted avg       0.66      0.67      0.65      1022



In [ ]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.72552783109405


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.7263603027457576, 0.6038023085857517, 0.6415050563236048, None)
